In [1]:
import pprint
import os
import sys
import numpy as np
from params import get_cfg
from rnn_base import RNN
from mem_sel_rnn import MemRNN
from NoEmbedRNN import OpSel
from NoEmbedRNN import MemSel
from NoEmbedRNN import RNN as oldRNN
from NoEmbedRNN import MemRNN  as oldMemRNN
from NoEmbedRNN import HistoryRNN
from ops import Operations
from session import *
from data_gen import *
import pickle
from functools import reduce

modelName = 'RNN'
old = True
func = 'np_avg_val-5ops'
name = 'one_both_hardmaxes_have_sat_sat_grad_out_NONREPRODUCEBLE/state_size_300-num_samples_3500-batch_size_100-learning_rate_0.001-epsilon_0.001-num_features_4-norm_True-softmax_sat_800-clip_False-share_state_True-rnns_same_state_False-state_fn_relu-pen_sofmax_True-smax_pen_r_0-seed_94169'
path = '/home/user/neural_program_synthesis/models/summaries/'+modelName+'/'+func+'/'+name+'/'
model_path = path+'/model'
#get the global configuration
cfg_path = path+'cfg.p'
cfg = pickle.load(open(cfg_path, 'rb'))
#instantiate containter with the operations avail for the selection
ops = Operations(cfg)
ops.ops = cfg["used_ops_obj"]
ops.num_of_ops = len(ops.ops)
#generate data 
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], cfg['seed'])
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])

if modelName == "RNN":
        #instantiante the mem selection RNN
        if old: mem = oldMemRNN(cfg, ops)
        else:   mem = MemRNN(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        if old: model = oldRNN(cfg, ops, mem)
        else:   model = RNN(cfg, ops, mem)
        res = restore_selection_matrixes2RNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)
elif modelName == "HistoryRNN":
        #instantiante the mem and op selection
        mem_sel = MemSel(cfg, ops)
        op_sel = OpSel(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        model = HistoryRNN(cfg, ops, mem_sel, op_sel)
        res = restore_selection_matrixes_HistoryRNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)

timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
[<tf.Variable 'RNN_mem/Params/W_mem:0' shape=(304, 300) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b_mem:0' shape=(300,) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/W2_mem:0' shape=(300, 5) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b2_mem:0' shape=(5,) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/W:0' shape=(304, 300) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b:0' shape=(300,) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/W2:0' shape=(300, 5) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b2:0' shape=(5,) dtype=float64_ref>]
norming the grads
grads are
[<tf.Tensor 'RNN_op/Grads/clip_by_global_norm/RNN_op/Grads/clip_by_global_norm/_0:0' shape=(304, 300) dtype=float64>, <tf.Tensor 'RNN_op/Grads/clip_by_global_norm/RNN_op/Grads/clip_by_global_norm/_1:0' shape=(300,) dtype=float64>, <tf.Tensor 'RNN_op/Grads/clip_by_global_norm/RNN_op/Grads/clip_by_glob

In [2]:
def print_errors(error_lst, name):
    print("")
    print("Total", reduce((lambda x, y: x + y), error_lst))
    for i, error in enumerate(error_lst):
        print(name + "[" + str(i) + "] err is " + str(error))

In [3]:
def print_matrix(total_error, matrix_lst, indeces = None):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        if indeces is not None and elem not in indeces: continue 
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            print(matrix[elem], end=" ")
    print("")

In [4]:
def print_ops_matrix(total_error, matrix_lst, ops_list, indeces = None):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        if indeces is not None and elem not in indeces: continue 
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            index = np.argmax(matrix[elem])
            op_name = ops_list[index].__name__
            print("[ "+op_name+" ]", end=" ")
    print("")

In [5]:
print_errors(res["total_loss_traind_train"], "sofmax_train_error")
print_errors(res["total_loss_traind_test"], "hardmax_train_error")
print_errors(res["total_loss_testd_train"], "softmax_test_error")
print_errors(res["total_loss_testd_test"], "hardmax_test_error")


Total 7993524.81498
sofmax_train_error[0] err is 328661.306732
sofmax_train_error[1] err is 262200.811589
sofmax_train_error[2] err is 440792.224826
sofmax_train_error[3] err is 328659.972569
sofmax_train_error[4] err is 330924.114628
sofmax_train_error[5] err is 396157.179254
sofmax_train_error[6] err is 354345.454382
sofmax_train_error[7] err is 308599.42085
sofmax_train_error[8] err is 326866.376788
sofmax_train_error[9] err is 356433.252503
sofmax_train_error[10] err is 377789.480559
sofmax_train_error[11] err is 331439.37839
sofmax_train_error[12] err is 307201.229494
sofmax_train_error[13] err is 380694.616109
sofmax_train_error[14] err is 357105.274483
sofmax_train_error[15] err is 380487.641601
sofmax_train_error[16] err is 377576.366968
sofmax_train_error[17] err is 301583.306727
sofmax_train_error[18] err is 335142.176585
sofmax_train_error[19] err is 416492.430005
sofmax_train_error[20] err is 322248.416841
sofmax_train_error[21] err is 336935.122773
sofmax_train_error[22] 

In [ ]:
#print_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test"][0])
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test"][0], cfg["used_ops_obj"])

In [ ]:
#print_matrix(res["total_loss_traind_test"][1], res["softmaxes_traind_test"][1])
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"])

In [ ]:
print_matrix(res["total_loss_testd_test"][0], res["outputs_testd_train"][0])

In [ ]:
wrong_res = np.nonzero(np.round(np.apply_along_axis(np_add, 1, res["outputs_testd_train"][0][4] - res["batchesY_test"][0]), 2))[0]
wrong_res 

In [ ]:
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test"][0], cfg["used_ops_obj"], wrong_res)

In [ ]:
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"], wrong_res)

In [ ]:
#print_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train_mem"][1])
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"])

In [ ]:
print_matrix(res["total_loss_testd_test"][0], res["outputs_testd_test_mem"][0])

In [ ]:
res["batchesX_test"][0]

In [ ]:
seed = 10000
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], seed)
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])
res = predict_form_sess(model, cfg, x_test[0:100,], last_hardmax_state_train, model_path, mode="hard")

In [ ]:
res - y_test[0:100,]

In [ ]:
y_test[0:100,]

In [ ]:
res

In [ ]:
len(res["softmaxes_traind_train"])